In [33]:
from pathlib import Path

import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

ROOT_PATH = Path("../../data/llff")

In [34]:
# Load reference poses
REFERENCE_PATH = ROOT_PATH / "deer_v6_debug_script_picked_blurry"
reference_poses = np.load(str(REFERENCE_PATH / "poses_bounds.npy"))
reference_poses.shape


(114, 17)

In [35]:
# Load poses that we expect to be a linear transformation of the reference poses
DESIRED_RESULT_PATH = ROOT_PATH / "deer_v6_debug"
desired_result_poses = np.load(str(DESIRED_RESULT_PATH / "poses_bounds.npy"))
desired_result_poses.shape

(202, 17)

In [36]:
# TODO: sort

# Match desired_result_poses to to reference poses by image that each corresponds to
reference_images = sorted([f for f in (REFERENCE_PATH / "images").iterdir()])
desired_result_images = sorted([f for f in (DESIRED_RESULT_PATH / "images").iterdir()])
assert len(reference_images) == reference_poses.shape[0]
assert len(desired_result_images) == desired_result_poses.shape[0]

indices_of_matched_images_from_desired = list(filter(
    lambda x: str(x[1].name) in list(map(lambda y: str(y.name), reference_images)),
    enumerate(desired_result_images)
)) # List comprehension
assert len(indices_of_matched_images_from_desired) == len(reference_images)

indices = list(map(lambda x: x[0], indices_of_matched_images_from_desired))
len(indices)

114

In [37]:
desired_result_poses = desired_result_poses[indices, :]
desired_result_poses.shape

(114, 17)

In [38]:
# Drop near/far bounds
if reference_poses.shape[-1] == 17:
    reference_poses = reference_poses[:, :-2]
if desired_result_poses.shape[-1] == 17:
    desired_result_poses = desired_result_poses[:, :-2]

reference_poses.shape, desired_result_poses.shape

((114, 15), (114, 15))

In [39]:
# Normalize
X = StandardScaler().fit_transform(reference_poses)
y = StandardScaler().fit_transform(desired_result_poses)

# Fit linear mdel
linear_model = LinearRegression().fit(X, y) # Use default values
predictions = linear_model.predict(X)

# Use squared error
mean_squared_error(X, y, squared=True)

0.26777061176434863

In [40]:
loss = mean_squared_error(X, y, squared=True, multioutput="raw_values") # good
loss.reshape((3, -1))

array([[8.12824882e-04, 1.32087882e-03, 8.72954584e-04, 8.73939239e-04,
        0.00000000e+00],
       [1.56504821e-03, 8.10295527e-04, 1.19404283e-03, 1.20442626e-03,
        0.00000000e+00],
       [1.63307841e-03, 5.70530234e-04, 2.54969905e-03, 3.15145841e-03,
        4.00000000e+00]])

In [43]:
# Drop img wh, focal length and normalize
X = StandardScaler().fit_transform(
    reference_poses.reshape((reference_poses.shape[0], 3, 5))[:, :, :-1].reshape((reference_poses.shape[0], -1))
)
y = StandardScaler().fit_transform(
    desired_result_poses.reshape((desired_result_poses.shape[0], 3, 5))[:, :, :-1].reshape((desired_result_poses.shape[0], -1))
)

# Fit linear mdel
linear_model = LinearRegression().fit(X, y) # Use default values
predictions = linear_model.predict(X)

# Use squared error
error = mean_squared_error(X, y, squared=True)
error, np.sqrt(error)

(0.0013799313721024158, 0.037147427530078256)

In [44]:
loss = mean_squared_error(X, y, squared=True, multioutput="raw_values") # good
loss.reshape((3, -1))

array([[0.00081282, 0.00132088, 0.00087295, 0.00087394],
       [0.00156505, 0.0008103 , 0.00119404, 0.00120443],
       [0.00163308, 0.00057053, 0.0025497 , 0.00315146]])